# HW\#7 Two-View Geometry



## Import Modules

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import scipy.signal
import scipy.linalg
import mae6292.tools as mae6292
import importlib

from mae6292.imshow import cv2_imshow

## Load Image and Feature Points

We load two images and the matched keypoints. They are visualized as follows.


In [ ]:
# load images
img1 = cv2.imread('data/0001.jpg')
img2 = cv2.imread('data/0002.jpg')

# load intrinsic paramters
K = np.array([[1379.74, 0, 760.35],[0, 1382.08, 503.41], [0,0,1]])
K1 = K;
K2 = K;

# load matches keypoints 
p1 = np.loadtxt('data/matches0001.txt')
p2 = np.loadtxt('data/matches0002.txt')
n = p1.shape[1]
# extend to homogeneous cooridinates
p1 = np.concatenate( (p1, np.ones((1,n))), axis=0 )
p2 = np.concatenate( (p2, np.ones((1,n))), axis=0 )

# visualization
fig, axes = plt.subplots(1, 2, dpi=120, figsize=(12,6))
axes[0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
axes[0].plot(p1[0,:],p1[1,:],'y.')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
axes[1].plot(p2[0,:],p2[1,:],'y.')
axes[1].axis('off')

## Problem 1 (Eight-Point Algorithm)

For the given $p_1, p_2, K_1, K_2$, compute the essential matrix according to the following steps. 

1. First, pixel coordinates are normalized by left-multiplying $K^{-1}$.
   Use [scipy.linalg.inv()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.inv.html)

2. Each row of $Q$ is given by $(\bar p_1^i \otimes \bar p_2^i)^T$. Use [np.kron()](https://numpy.org/doc/stable/reference/generated/numpy.kron.html)

3. Decompose $Q = US V^T$, and $\mathrm{vec}(E)$ corresponds to the last column of $V$. Use [scipy.linalg.svd()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svd.html)

4. Reshape $\mathrm{vec}(E)$ into a 3 by 3 matrix $E$.  

And verity that 
```
E=[[-2.32658231e-04 -2.30066653e-02 -1.38566261e-03]
 [ 1.61584860e-01  1.70952962e-03  6.90754078e-01]
 [-1.94754336e-03 -7.04422370e-01  5.14852933e-04]]
```

## Problem 2 (Triangulation)

See the notes for triangulartion in stereo vison. Let the perspective projection matrix be $M_1 = K_1[I_{3\times3}|0]$ and $M_2 = K_2[R|T]$. 
For given $p_1, p_2, M_1, M_2$, generate a function that returns the 3D world cooridiantes $P^i$ of features. 
We need this function to resolve the ambiguity in extracting $(R,T)$ from $E$.

The homogeneous coordinates of the 3D point $P^i$ belongs to the null space of
\begin{align*}
null \begin{bmatrix} \hat p^i_1 M_1 \\ \hat p^i_2 M_2 \end{bmatrix}_{6\times 4}
\end{align*}
The hat map has been implemented at `mae6292.hat()`. When defining $M$, one can use [np.concatenate()](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html)

In [ ]:
def triangulation(p1, p2, M1, M2):
    # Input
    # p1, p2: 3 by n pixel homogeneous coordinates
    # M1, M2, 3 by 4 perspective projection matrix
    # Output
    # P: 3 by n (nonhomogeneous) world coordinate
    assert(p1.shape[0]==3 and p2.shape[0]==3 and p1.shape[1]==p2.shape[1] )
    
    n=p1.shape[1]


    
    
    
    
    
    
    
    
    return P

Verify the above function with random data. Since we did not use any special structure of $M_1$ and $M_2$ in the above expression, they can be generated randomly. The following codes generate random inputs, and the resulting error is supposed to be at the order of $10^{-14}-10^{-15}$.

In [ ]:
n_test = 10
P_test = np.random.rand(4,n_test)
M1_test = np.random.rand(3,4)
M2_test = np.random.rand(3,4)
p1_test = M1_test@P_test
p2_test = M2_test@P_test

P_test_new = triangulation( p1_test, p2_test, M1_test, M2_test)

print('Recontrsuction error for P',scipy.linalg.norm(P_test[0:3,:]/P_test[3,:]-P_test_new))

## Problem 3 (Extract $(R,T)$)

### (a) Compute four combinations of $(R,T)$
Let the singular value decomposition of the essential matrix be $E = USV^T$. The corresponding relative pose $(R,T)$ can be extracted by
\begin{align*}
R & = \det(UV^T) U W V^T, \text{ or } \det(UV^T) UW^TV^T,\\
T & = \pm U e_3,
\end{align*}
where 
\begin{align*}
W =
\begin{bmatrix} 0 &  1 & 0 \\
        - 1 & 0 & 0 \\
    0 & 0 & 1 \end{bmatrix}.
\end{align*}

As such there are four possible combinations of $(R,T)$. Compute all of four combinations of $(R,T)$.



In [ ]:
R = np.zeros((3,3,4))
T = np.zeros((3,4))
W = np.array( [[0,1,0], [-1,0,0], [0,0,1]])



## (b) Identify the true $(R,T)$

Among the four combinations, only one guarantees that the 3D points are in front of both cameras. Using the function `triangulation`, compute $P^i$ for each combination of $(R,T)$, and count the number of feature points that have negative $Z_w$ value. The particular combination of $(R,T)$ with the mininum number of depth is the correct one.

One may use [np.where()](https://numpy.org/doc/stable/reference/generated/numpy.where.html) and [np.argmin()](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html)

Verity that
```
T= [[ 0.9994653 ]
 [ 0.00197824]
 [-0.03263744]]
```

In [ ]:
M1 = K1@np.concatenate( (np.identity(3), np.zeros((3,1))), axis=1 )
N_negative_depth = np.zeros(4)





print('T=',T)

### (c) Triangulation

Using the extracted $(R,T)$, triangulate $P^i$.

Verify 
```
P^0= [-0.14620011 -1.67792297  5.06569238]
```

### (d) Reprojection Error

Using the extracted $R,T$ and the triangulated $P$, project $P$ back to the image plane with
\begin{align*}
\lambda_1^i \tilde p^i_1 = M_1 P,\quad \lambda_2^i \tilde p^i_2 = M_2 P.
\end{align*}
De-homogenize $\tilde p^i_1$ and $\tilde p^i_2$ to obtain $(\tilde u^i_1, \tilde v^i_1)$ and $(\tilde u^i_2, \tilde v^i_2)$ respectively. 

Let $(u_1^i, v_1^i)$ and $(u_2^i, v_2^i)$ be the original pixel cooridantes.
The reprojection error is
\begin{align*}
\mathrm{err} = \sum_{i=1}^n (u^i_1 - \tilde u^i_1)^2 + (v^i_1-\tilde v^i_1)^2 + 
(u^i_2 - \tilde u^i_2)^2 + (v^i_2-\tilde v^i_2)^2
\end{align*}

Compute the reprojection error. You may see [SSD in numpy/scipy](https://stackoverflow.com/questions/2284611/sum-of-square-differences-ssd-in-numpy-scipy/2284634)

Verify that
```
error= 8714.388063741104
```

### (e) Visualization

Visualize the 3D scene by the following commands. You may turn the viewpoint such that the reconstructed scene resemsbles the images above, save it into `prob2.png` using the command 
```
plt.savefig('prob2.png')
```

In [ ]:
def draw_frame(ax, R, T, length=1):
    C = -R.T@T.flatten()
    colors=('r','g','b')
    for i in range(3):
        ax.plot((C[0],C[0]+length*R[i,0]),(C[1],C[1]+length*R[i,1]),(C[2],C[2]+length*R[i,2]), colors[i])

In [ ]:

fig = plt.figure(dpi=150)
ax = fig.add_subplot(projection='3d')

ax.plot(P[0,:],P[1,:],P[2,:],'b.')
ax.axes.set_xlim3d(-3,3)
ax.axes.set_ylim3d(-3,3) 
ax.axes.set_zlim3d(-1,6)
ax.set_xlabel('x')ax.set_ylabel('y')
ax.set_zlabel('z')

draw_frame(ax, R,T)
draw_frame(ax, np.identity(3), np.zeros((3,1)) )

ax.view_init(-90,-90) # front view


## How to Submit

Add
1. `prob2.png`

to your commit and push it to github.